In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch

import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("/kaggle/input/computed-tomography-ct-of-the-abdomen/ct_abdomen.csv")

In [ ]:
EPOCHS = 10
LR = 0.1
BATCH = 32
IM_SIZE= 224
STEP = 5
GAMMA = 0.1
NUM_CLASSES = df["type"].nunique()

In [ ]:
df.drop(["dcm"], axis=1, inplace=True)

In [ ]:
le = LabelEncoder()
df["type"] = le.fit_transform(df.iloc[:, -1].values)

In [ ]:
base_dir = "/kaggle/input/computed-tomography-ct-of-the-abdomen/files/files"
df["jpg"] = df["jpg"].apply(lambda x: os.path.join(base_dir, x[1:]))

In [ ]:
train, tests = train_test_split(df, random_state=42, test_size=0.2)
val, test = train_test_split(tests, random_state=42, test_size=0.5)

In [ ]:
X = []
Y = []
for i in range(len(train)):
    img = Image.open(train.iloc[i, 0])
    hr = img.transpose(method = Image.FLIP_LEFT_RIGHT)
    vr = img.transpose(method = Image.FLIP_TOP_BOTTOM)
    X.append(np.array(img))
    Y.append(train.iloc[i, -1])
    X.append(np.array(hr))
    Y.append(train.iloc[i, -1])
    X.append(np.array(vr))
    Y.append(train.iloc[i, -1])

In [ ]:
class Clod(torch.nn.Module):
    def __init__(self, data, target=None, transform=None):
        super(Clod, self).__init__()
        self.data = data
        self.transform = transform
        self.target = target
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, x):
        if self.target is not None:
            image, label = self.data[x], self.target[x]
        else:
            image, label = self.data.iloc[x, 0], self.data.iloc[x, -1]
            image = Image.open(image).convert('RGB')
            image = np.array(image)
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(),
                               transforms.ToTensor(),
                               transforms.Resize((IM_SIZE, IM_SIZE)),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])])

In [ ]:
train_ds = Clod(X, Y, transform)
val_ds = Clod(val, None, transform)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
num_ftrs = model.fc.in_features
for param in model.parameters():
    param.requires_grad_ = False
model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)
model.fc.requires_grad_ = True

In [ ]:
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=GAMMA, step_size=STEP)

In [ ]:
best_model = deepcopy(model)
best_acc = 0.0

acc_train = []
acc_val = []
loss_train = []
loss_val = []

for i in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    train_total = 0
    
    for data, label in train_dl:
        optimizer.zero_grad()
        if torch.cuda.is_available():
            data, label = data.cuda(), label.cuda()
            
        out = model(data)
        loss = criterion(out, label)
        train_loss += loss.item()
        train_acc += (out.argmax(1) == label).sum().item()
        train_total += out.size(0)
        loss.backward()
        optimizer.step()
        
    train_loss /= train_total
    train_acc /= train_total
        
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    val_total = 0
    
    with torch.no_grad():
        for data, label in val_dl:
            if torch.cuda.is_available():
                data, label = data.cuda(), label.cuda()
                
            out = model(data)
            loss = criterion(out, label)
            val_loss += loss.item()
            val_acc += (out.argmax(1) == label).sum().item()
            val_total += out.size(0)
            
    val_acc /= val_total
    val_loss /= val_total
    acc_train += [train_acc]
    acc_val += [val_acc]
    loss_train += [train_loss]
    loss_val += [val_loss]
    
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = deepcopy(model)
        
    print("Epoch {} train loss {} acc {} val loss {} acc {}".format(i, train_loss, train_acc, val_loss, val_acc))
    scheduler.step()

In [ ]:
epochs = list(range(1, EPOCHS+1))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axes[0].plot(epochs, loss_train)
axes[0].plot(epochs, loss_val)
axes[0].set_title("Training and Validation loss")
axes[0].legend(["Training", "Validation"])
axes[1].plot(epochs, acc_train)
axes[1].plot(epochs, acc_val)
axes[1].set_title("Training and Validation accuracies")
axes[1].legend(["Training", "Validation"])
plt.suptitle("ResNet50 best accuracy: {}%".format(round(best_acc*100, 2)))
plt.tight_layout()
plt.show()

In [ ]:
def predict(value):
    image = Image.open(value).convert("RGB")
    image = np.array(image)
    image = transform(image)
    image = image.reshape([1, image.shape[0], image.shape[1], image.shape[2]])
    if torch.cuda.is_available():
        image = image.cuda()
    best_model.eval()
    with torch.no_grad():
        out = model(image)
    index = out.argmax(1).item()
    return index

In [ ]:
predicted = []
gt = []
for i in range(len(test)):
    predicted += [predict(test.iloc[i, 0])]
    gt += [test.iloc[i, -1]]

In [ ]:
score = accuracy_score(predicted, gt)
report = classification_report(predicted, gt)
cm = confusion_matrix(predicted, gt)
print("Test image score: {}%".format(round(score*100, 2)))
print("Report")
print(report)
sns.heatmap(cm, annot=True)
plt.show()

In [ ]:
predicted = le.inverse_transform(predicted)
gt = le.inverse_transform(gt)

In [ ]:
index = 0
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 12))
for i in range(4):
    for j in range(4):
        im = plt.imread(test.iloc[index, 0])
        axes[i][j].imshow(im)
        axes[i][j].set_title("Predicted: {}\nActual: {}".format(predicted[index], gt[index]))
        index += 1
        
plt.tight_layout()
plt.show()